In [ ]:
from dep_tools.loaders import OdcLoader
from dep_tools.searchers import PystacSearcher
from odc.stac import configure_s3_access
from ldn.geomad import (
    GeoMADLandsatProcessor,
    USGS_CATALOG,
    USGS_COLLECTION
)
from dask.distributed import Client as DaskClient

from ldn.grids import get_gridspec

In [ ]:
# Reload functions
%load_ext autoreload
%autoreload 2

In [ ]:
# Make sure we can access S3
_ = configure_s3_access(cloud_defaults=True, requester_pays=True)

In [ ]:
decimated = False

datetime = "2025"
xy_chunk_size = 1000
geomad_threads = 10
scale = 0.0000275
offset = -0.2
drop_vars = ["qa_pixel"]

tile_id = "136,142"

grid = get_gridspec()
tile_index = tuple(int(i) for i in tile_id.split(","))
geobox = grid.tile_geobox(tile_index)

if decimated:
    geobox = geobox.zoom_out(10)

geobox.explore()


In [ ]:
search_kwargs = {
    "query": {"landsat:collection_category": {"in": ["T1"]}}
}

searcher = PystacSearcher(
    catalog=USGS_CATALOG, collections=[USGS_COLLECTION], datetime=datetime, **search_kwargs
)

items = searcher.search(geobox)

print(f"Found {len(items)} items")

In [ ]:
loader = OdcLoader(
    bands=["qa_pixel", "red", "green", "blue"],
    chunks={"x": 2048, "y": 2048, "time": 1},
    groupby="solar_day",
    fail_on_error=False,
)

data = loader.load(items, geobox)
data

In [ ]:
processor = GeoMADLandsatProcessor(
    geomad_options=dict(
        work_chunks=(xy_chunk_size, xy_chunk_size),
        num_threads=geomad_threads,
        maxiters=100,
        scale=scale,
        offset=offset,
        nodata=0
    ),
    min_timesteps=5,
    drop_vars=drop_vars
)

# Do the processing using a local dask cluster
with DaskClient(n_workers=1, threads_per_worker=32, memory_limit="16GB"):
    output_data = processor.process(data)

output_data

In [ ]:
output_data.odc.explore()

In [ ]:
# End-to-end test an item
from pystac import Item
from odc.stac import load

item = Item.from_file("https://data.ldn.auspatious.com/ausp_ls_geomad/0-0-0/136/142/2025/ausp_ls_geomad_136_142_2025.stac-item.json")
data = load([item])
data

In [ ]:
data.odc.explore()